## Day 4
Adding to the analysis performed on Day 3, how many times have teams been in the top 3 in terms of points scored against at home?

In [ ]:
import pandas as pd
from os import path
import sqlite3
import nfl_data_py as nfl


# Create database connection
conn = sqlite3.connect('../data/db/database.db')

In [ ]:
# 2021 season was the only season with an uneven number of home games
# Analysis will be done on seasons 1999-2000

query = """
WITH data AS (
    SELECT
        game_id,
        season,
        game_type,
        week,
        away_team,
        away_score,
        home_team,
        home_score 
    FROM schedules
    WHERE game_type = 'REG'
)
SELECT DISTINCT season, count(*) AS num_home_games
FROM data
GROUP BY home_team, season
HAVING num_home_games <> 8
"""

pd.read_sql(query, conn)

,season,num_home_games
0,2021,9


In [ ]:
query = """
WITH data AS (
    SELECT
        game_id,
        season,
        game_type,
        week,
        away_team,
        away_score,
        -- Need to update data to account for 3 teams that switched cities between 1999-2020
        CASE
            WHEN home_team = 'STL' THEN 'LAR'
            WHEN home_team = 'SD' THEN 'LAC'
            WHEN home_team = 'OAK' THEN 'LV'
            ELSE home_team
        END AS home_team,
        home_score 
    FROM schedules
    WHERE season <> 2021
        AND game_type = 'REG'
), rankings AS (
    SELECT
        season,
        home_team AS team,
        AVG(away_score) AS avg_pa_at_home,
        DENSE_RANK() OVER(PARTITION BY season ORDER BY AVG(away_score) ASC) AS best_to_worst,
        DENSE_RANK() OVER(PARTITION BY season ORDER BY AVG(away_score) DESC) AS worst_to_best
    FROM
        data
    GROUP BY
        season, home_team
), top3 AS (
    SELECT
        season,
        team,
        ROUND(avg_pa_at_home,2) AS avg_pa_at_home,
        best_to_worst
    FROM rankings
    WHERE best_to_worst <= 3
), bottom3 AS (
    SELECT
        season,
        team,
        ROUND(avg_pa_at_home,2) AS avg_pa_at_home,
        worst_to_best
    FROM rankings
    WHERE worst_to_best <= 3
)
SELECT
    team, 
    COUNT(*) AS num_seasons
FROM bottom3
GROUP BY team
ORDER BY COUNT(*) DESC
"""

pd.read_sql(query, conn)

,team,num_seasons
0,TEN,6
1,LAR,6
2,CIN,6
3,NO,5
4,MIN,4
5,LV,4
6,DAL,4
7,ATL,4
8,TB,3
9,KC,3
